In [1]:
import tools

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
print(os.getenv("LANGSMITH_ENDPOINT"))

https://api.smith.langchain.com


In [3]:
import os

import os
from langchain.chat_models import init_chat_model



llm = init_chat_model(
    "azure_openai:gpt-4.1-mini",
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version="2024-12-01-preview"
)

agent_tools = [tools.get_all_plants, tools.get_bloom_data]
llm_with_tools = llm.bind_tools(agent_tools)

In [4]:
from utils import make_specialist

plants = tools.get_all_plants().split(" ")
print(plants)
#create specialists
specialists = {plant: make_specialist(plant, llm) for plant in plants}
[specialist.name for specialist in specialists.values()]

['apfel', 'birke', 'erle', 'hasel', 'winterraps', 'winterroggen', 'esche', 'forsythie', 'huflattich', 'loewenzahn', 'schneegloeckhen', 'beifuss']


['apfel_agent',
 'birke_agent',
 'erle_agent',
 'hasel_agent',
 'winterraps_agent',
 'winterroggen_agent',
 'esche_agent',
 'forsythie_agent',
 'huflattich_agent',
 'loewenzahn_agent',
 'schneegloeckhen_agent',
 'beifuss_agent']

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
import json
from utils import get_tokens
#base_prompt = 'Antworte in JSON und folge dem Schema: { "type": "object", "properties": { "response": { "type": "boolean" }, "reasoning": { "type": "string" } }, "required": [ "response", "reasoning" ] }'
with open("datasets/dataset_850.json", "r") as file:
    dataset = json.load(file)


responses = []
plant_count = 0
for agent in specialists.values():
    count = 0
    for data in dataset:
        sentence = data["sentence"]
        prompt = [HumanMessage(content=sentence)]
        print(f"Processing: {sentence}")
        try:
            messages = agent.invoke({"messages": prompt})
        except Exception as e:
            print(f"Error processing {sentence}: {e}")
            responses.append({
                "id": data["id"],
                "plant": data["plant"],
                "city": data["city"],
                "date": data["date"],
                "sentence": sentence,
                "response": "error",
                "reasoning": str(e),
                "tokens": 0
            })
        for m in messages['messages']:
            m.pretty_print()
        response = messages["messages"][-1].content
        cleaned_response = response.strip("```json").strip("```").strip()
        response_json = json.loads(cleaned_response)
        tokens = get_tokens(messages)
        responses.append({
            "id": data["id"],
            "plant": data["plant"],
            "city": data["city"],
            "date": data["date"],
            "sentence": sentence,
            "response": response_json["response"],
            "reasoning": response_json["reasoning"],
            "tokens": tokens
        })
        count += 1
        print("Processed:", count, "/", len(dataset), "responses collected:", len(responses))
        print("Plant count:", plant_count, "/", len(specialists))


    with open(f"results/{agent.name}_850.json", "w") as file:
        json.dump(responses, file, indent=4)

Processing: Haben sich bis 29.04.2021 in Nürnberg bereits Blüten an der Apfel gezeigt?
================================ Human Message =================================

Haben sich bis 29.04.2021 in Nürnberg bereits Blüten an der Apfel gezeigt?
================================== Ai Message ==================================
Name: apfel_agent
Tool Calls:
  get_bloom_data_apfel (call_XdS5vx3O9Y496Y9wDCOc3NLt)
 Call ID: call_XdS5vx3O9Y496Y9wDCOc3NLt
  Args:
    year: 2021
================================= Tool Message =================================
Name: get_bloom_data_apfel

{"data": {"Jahr": {"4": 2021}, "Gebietsmittel": {"4": "01.05. "}, "Abweichung vom Mittel": {"4": "5 Tage"}}, "info": "Tabelle mit den mittleren Beobachtungsterminen und den Abweichungen vom Mittel der Phase \"Apfel: Blüh-Beginn\" im Gebiet Deutschland für verschiedene Jahre. Positive Abweichungen bedeuten, dass die Phase im entsprechendem Jahr später aufgetreten ist.\n\n© Deutscher Wetterdienst, erstellt 23.05.2025

KeyboardInterrupt: 